In [1]:
from util_all import * 

## Loading the DATA

In [2]:
# Loading all the data from Matlab file
data=scipy.io.loadmat(f'data/features_matrix.mat')
labels= np.squeeze(data['labels'])
labels_for_ML = 10**labels
Y = pd.DataFrame({'y': labels_for_ML})
features = data['features']
cycling_conditions = data['cycling_conditions']
features_names0 = data['features_names']
features_names = [subarray[0] for subarray in features_names0[0]]
cycling_conditions_names0 = data['cycling_conditions_names']
cycling_conditions_names = [subarray[0] for subarray in cycling_conditions_names0[0]]

#Concatenating all features in one big array of dim num_cells x num_features
all_features = features[:,:,0]
for i in range(1,3):
    all_features=np.concatenate((all_features,features[:,:,i]),axis=1)
    
# Creating features names (handles for features df column and metadata df )
classes = ['charge', 'full', 'discharge']
all_features_names = [my_class+'_'+s  for my_class in classes for s in features_names]

# Creating the Features Dataframes
all_features_df = pd.DataFrame(all_features, columns = all_features_names)
cycling_conditions_df = pd.DataFrame(cycling_conditions, columns = cycling_conditions_names)

In [3]:
#creating metadata Dataframe for all features
features_metadata_df = pd.DataFrame(index = all_features_names)
for name, row in features_metadata_df.iterrows():
    features_metadata_df.loc[name,'type']=get_feature_type(name)
    features_metadata_df.loc[name,'class']=get_feature_class(name)
    features_metadata_df.loc[name,'stream']=get_feature_stream(name)
    features_metadata_df.loc[name,'num_cycles_needed']=int(name[name.find('y')+9:name.find('y')+12]) 
    my_min, my_max = get_min_max_percentile(name)
    features_metadata_df.loc[name,'min_percentile']=my_min
    features_metadata_df.loc[name,'max_percentile']=my_max
    features_metadata_df.loc[name,'Pearson']=abs(pearsonr(all_features_df[name],labels_for_ML)[0])
    features_metadata_df.loc[name,'Log_Pearson']=abs(pearsonr(all_features_df[name],labels)[0])
    
#creating metadata Dataframe for cycling conditions
cycling_conditions_metadata_df = pd.DataFrame(data = [0,0,0,0],index=cycling_conditions_names,columns=['class'])
for name, row in cycling_conditions_metadata_df.iterrows():
    cycling_conditions_metadata_df.loc[name,'Pearson']=abs(pearsonr(cycling_conditions_df[name],labels_for_ML)[0])
    cycling_conditions_metadata_df.loc[name,'Log_Pearson']=abs(pearsonr(cycling_conditions_df[name],labels)[0])
    


## Recovering protocols and exclude protocol of interest

In [4]:
file = open("data/dic_protocol_to_cell_idx.json")
dic_protocell = json.load(file)

In [6]:
lengths = []
for key in dic_protocell.keys():
    lengths.append(len(dic_protocell[key]))

In [7]:
protocols = pd.DataFrame(index = list(dic_protocell.keys()),columns = ['repeats','mean','std'])
for name in dic_protocell.keys():
    protocols.loc[name,'repeats']=len(dic_protocell[name])
    protocols.loc[name,'mean']=round(np.mean(Y.loc[dic_protocell[name]].values))
    protocols.loc[name,'std']=round(np.std(Y.loc[dic_protocell[name]].values))


## Tranferability study: excluding one protocol from training and validation

In [8]:
types =['charge','full','discharge']
time_regions = [150] #comparing charging, full, and discharging models performance. We want a fair comparison so featurization from 141-150 is preferred

#Getting a list of 3xlen(time_regions) datasets, 1 per type and time_region
metadata_per_class_per_TR = [features_metadata_df[(features_metadata_df['type']==x)&(features_metadata_df['num_cycles_needed']==y)] for x in types for y in time_regions]  
datasets = [all_features_df[meta_x.index.values.tolist()] for meta_x in metadata_per_class_per_TR]

#creating handles for which dataset is which:
my_dataset_order = np.array([[x,y] for x in types for y in time_regions])
my_types = my_dataset_order[:,0]
my_time_regions = my_dataset_order[:,1]

# Initializing my results DF
my_columns = ['train MAPE','val MAPE','test MAPE','train RMSE','val RMSE','test RMSE','train MAE','val MAE','test MAE']

In [9]:
long_protocols = protocols[protocols['repeats']>=5]
protocol = list(long_protocols.index)[1]
ruleout_list = dic_protocell[protocol]

In [ ]:
nb_TTS = 10
prot_results = np.empty((nb_TTS,9,0))

for i in range(len(datasets)):
# for i in range(2):
    X=datasets[i]
    X_trainval,X_test, Y_trainval,Y_test = ruleout_split(X,Y,ruleout_list)

    #Initializing results array
    RF_trainvaltest_results =  np.empty((0,9))

    #looping over 10 Train Test Splits:
    for rdstate in range(nb_TTS):
        print(f'the dataset is {i} and the TTS is {rdstate}')
    ## Splitting into train (train/val) and test sets
        X_train, X_val, Y_train, Y_val = train_test_split(X_trainval, Y_trainval, test_size = 0.2,random_state = rdstate)

        ## Make unified dataframes for train and test
        df_train = Y_train.join(X_train)
        df_val = Y_val.join(X_val)

        Y_col = ['y']
        X_col =  X.columns.tolist()
    
        #finding best RF
        opt_model, X_train_scaled, Y_train_scaled, X_scaler, Y_scaler = make_cv_model_object(df_train,
                                                 Y_col=Y_col,
                                                 X_col=X_col,
                                                cv_splits = 10,                                       
                #                                  split_lists=split_lists,
                                                 model=RandomForestRegressor(random_state=0),
                                                 model_hyperparams={'n_estimators': [40,80,160],
                                                                    'min_samples_leaf':[2,4],#[1,2,4,8]
                                                                    'min_samples_split':[2,4,8]})

        #Best RF:
        best_RF= opt_model.best_estimator_
        best_RF.fit(X_train_scaled,Y_train_scaled.values.ravel())

        RF_trainval_iter= plot_train_test_model_predictions(best_RF,
                                              X_train_scaled = X_train_scaled, X_test = X_val, X_scaler = X_scaler, 
                                             Y_train = Y_train, Y_test = Y_val, Y_scaler = Y_scaler,
                                                X_col=X_col,Y_col=['y'],
                                             plot_bounds = [0,2000], plot=False)[0:6]

        RF_trainval_iter= np.array(RF_trainval_iter).reshape((1,-1))

        #We now look into how the model performs on ruled-out cells, aka Test set:
        #to plot parity plot only once per dataset, we print only during the first TTS.
        if rdstate ==0:
            myFalse = False #set to True to plot parity plot 
        else: 
            myFalse= False
        _,RF_test_MAPE,_,RF_test_RMSE,_,RF_test_MAE = plot_train_test_model_predictions(best_RF,
                                          X_train_scaled = X_train_scaled, X_test = X_test, X_scaler = X_scaler, 
                                         Y_train = Y_train, Y_test = Y_test, Y_scaler = Y_scaler,
                                            X_col=X_col,Y_col=['y'],
                                         plot_bounds = [0,2000], plot=myFalse)[0:6]

        #Finally, we concatenate the train, val, test errors and add them to our results df
        RF_trainvaltest_iter=np.insert(RF_trainval_iter,[2,4,6], [RF_test_MAPE,RF_test_RMSE,RF_test_MAE]).reshape((1,-1))
        RF_trainvaltest_results = np.append(RF_trainvaltest_results,RF_trainvaltest_iter,axis=0)
    RF_trainvaltest_results = np.expand_dims(RF_trainvaltest_results,axis=-1) #of shape (nb_TTS,9,1)
    prot_results = np.append(prot_results,RF_trainvaltest_results,axis=2)

In [12]:
#Extracting means and std for one protocol
prot_means = np.mean(prot_results,axis =0)
prot_std = np.std(prot_results,axis=0)
prot_means= prot_means.T
prot_std = prot_std.T

#Function to build digestible dataFrames
def construct_df (array,my_columns,my_types):
    df = pd.DataFrame(array,columns=my_columns)
    df['type']=my_types
    return df

# Constructing digestible dataFrame
my_columns = ['train MAPE','val MAPE','test MAPE','train RMSE','val RMSE','test RMSE','train MAE','val MAE','test MAE']
prot_means_df = construct_df(prot_means,my_columns,types)
prot_std_df = construct_df(prot_std,my_columns,types)
prot_means_df= prot_means_df.drop(index =1)
prot_std_df = prot_std_df.drop(index =1)
